In [95]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

citations = pd.read_csv("cora.cites",
    sep="\t",
    header=None,
    names=["target", "source"],
)
print("Citations shape:", citations.shape)

column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv("cora.content", sep="\t", header=None, names=column_names,
)
print("Papers shape:", papers.shape)


Citations shape: (5429, 2)
Papers shape: (2708, 1435)


In [96]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

train_data, test_data = [], []

for _, group_data in papers.groupby("subject"):
    # Select around 50% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.5
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)



Train data shape: (1322, 1435)
Test data shape: (1386, 1435)


In [97]:
train_data

,paper_id,term_0,term_1,term_2,term_3,term_4,term_5,term_6,term_7,term_8,...,term_1424,term_1425,term_1426,term_1427,term_1428,term_1429,term_1430,term_1431,term_1432,subject
305,2032,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4
1261,1441,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1329,70,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,6
981,694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1926,987,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
1275,613,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,5
1578,1142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5
698,483,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2


In [3]:
def create_MLP(capas_internas, dropout_rate, name=None):
    capas = []

    for capa_interna in capas_internas:
        capas.append(layers.BatchNormalization())
        capas.append(layers.Dropout(dropout_rate))
        capas.append(layers.Dense(capa_interna, activation=tf.nn.gelu))

    return keras.Sequential(capas, name=name)


In [98]:
feature_names = set(papers.columns) - {"paper_id", "subject"}
num_features = len(feature_names)
num_classes = len(class_idx)

# Create train and test features as a numpy array.
x_train = train_data[feature_names].to_numpy()
x_test = test_data[feature_names].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data["subject"]
y_test = test_data["subject"]


/var/folders/gf/12lsjk9d5mn0gdyhlk_xd0zm0000gn/T/ipykernel_8662/1433920676.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x_train = train_data[feature_names].to_numpy()
/var/folders/gf/12lsjk9d5mn0gdyhlk_xd0zm0000gn/T/ipykernel_8662/1433920676.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x_test = test_data[feature_names].to_numpy()


## La clase modelo en Keras

Para definir capas en una GNN tenemos que extender la clase *model* en Keras. Para orientarnos con esta funcionalidad vamos a crear primero una red neuronal básica que nos permita comparar resultados. La idea es que esta red básica sirva para predecir los temas de los papers usando solamente los features de cada paper (nodo en el grafo), y no sus conexiones. 


In [5]:
class BaseClassifier(tf.keras.Model):
    def __init__(
        self,
        num_classes,
        capas_internas = [32,32],
        dropout_rate=0.5,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(BaseClassifier, self).__init__(*args, **kwargs)

        
        ### Definimos las capas de nuestro modelo como atributos. 
        ## Una capa que preprocese node_features y los lleve a una dimensionalidad mas pequeña
        self.preprocesador = create_MLP(capas_internas, dropout_rate, name="preprocesador")

        ## Tres capas adicionales 
        
        self.capa1 = create_MLP(capas_internas, dropout_rate, name="capa1")
        self.capa2 = create_MLP(capas_internas, dropout_rate, name="capa2")
        self.capa3 = create_MLP(capas_internas, dropout_rate, name="capa3")

        ## La ultima capa que llegue a num_clases 

        self.clasificador = layers.Dense(units=num_classes, name="clasificador")
        
    def call(self,input_features):
            
        i = self.preprocesador(input_features)
        c1 = self.capa1(i)
            
            #skip connection
        s1 = c1 + i 
            
            #otra capa, skip, capa, skip
        c2 = self.capa2(s1)
        s2 = c2 + s1 
        c3 = self.capa3(s2)
        s3 = c3 + s2
        

        return self.clasificador(s3)
        
        
        

In [26]:
BASE = BaseClassifier(
    num_classes=7,
    name="modelo_base",
)



In [27]:
BASE.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

    # Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_acc", patience=50, restore_best_weights=True
)
    # Fit the model.
history = BASE.fit(
    x=x_train,
    y=y_train,
    epochs=300,
    batch_size=256,
    validation_split=0.15,
    callbacks=[early_stopping],
)

Epoch 1/300
5/5 [==============================] - 2s 47ms/step - loss: 4.3904 - acc: 0.1244 - val_loss: 1.8880 - val_acc: 0.2585
Epoch 2/300
5/5 [==============================] - 0s 10ms/step - loss: 2.5492 - acc: 0.2150 - val_loss: 1.8804 - val_acc: 0.2585
Epoch 3/300
5/5 [==============================] - 0s 8ms/step - loss: 2.3103 - acc: 0.3057 - val_loss: 1.8634 - val_acc: 0.2585
Epoch 4/300
5/5 [==============================] - 0s 8ms/step - loss: 2.0766 - acc: 0.3066 - val_loss: 1.8615 - val_acc: 0.2585
Epoch 5/300
5/5 [==============================] - 0s 8ms/step - loss: 1.9198 - acc: 0.3256 - val_loss: 1.8545 - val_acc: 0.2927
Epoch 6/300
5/5 [==============================] - 0s 9ms/step - loss: 1.7638 - acc: 0.3446 - val_loss: 1.8332 - val_acc: 0.3220
Epoch 7/300
5/5 [==============================] - 0s 7ms/step - loss: 1.7084 - acc: 0.3661 - val_loss: 1.8026 - val_acc: 0.2780
Epoch 8/300
5/5 [==============================] - 0s 9ms/step - loss: 1.6525 - acc: 0.3998 - v

5/5 [==============================] - 0s 8ms/step - loss: 0.4071 - acc: 0.8575 - val_loss: 0.9951 - val_acc: 0.6634
Epoch 65/300
5/5 [==============================] - 0s 9ms/step - loss: 0.4356 - acc: 0.8437 - val_loss: 1.0242 - val_acc: 0.6439
Epoch 66/300
5/5 [==============================] - 0s 7ms/step - loss: 0.4041 - acc: 0.8566 - val_loss: 0.9924 - val_acc: 0.6683
Epoch 67/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4222 - acc: 0.8463 - val_loss: 0.9067 - val_acc: 0.6878
Epoch 68/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4081 - acc: 0.8575 - val_loss: 0.9090 - val_acc: 0.6927
Epoch 69/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4244 - acc: 0.8549 - val_loss: 0.9147 - val_acc: 0.6829
Epoch 70/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4012 - acc: 0.8601 - val_loss: 0.9574 - val_acc: 0.6732
Epoch 71/300
5/5 [==============================] - 0s 8ms/step - loss: 0.3794 - acc: 0.8679 - val_loss

In [28]:
BASE.summary()

Model: "modelo_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocesador (Sequential)  (None, 32)                52804     
                                                                 
 capa1 (Sequential)          (None, 32)                2368      
                                                                 
 capa2 (Sequential)          (None, 32)                2368      
                                                                 
 capa3 (Sequential)          (None, 32)                2368      
                                                                 
 clasificador (Dense)        multiple                  231       
                                                                 
Total params: 60,139
Trainable params: 56,825
Non-trainable params: 3,314
_________________________________________________________________


In [34]:
BASE.evaluate(x=x_test, y=y_test)



43/43 [==============================] - 0s 1ms/step - loss: 0.7519 - acc: 0.7539


[0.7518908381462097, 0.753903329372406]

## Las GNN. 

El primer paso es crear una matriz de adyacencia en formato numpy, que es lo que vamos a necesitar para pasarselo a tensorflow. Por razones de formato, es mejor usar una representación esparsa, en forma de lista de pares. 

In [76]:
#Matriz en forma de lista de pares
edges = citations[["source", "target"]].to_numpy().T

#Codigo para agregar peso a cada arista, por ahora son puros 1s, todas valen lo mismo. 
edge_weights = tf.ones(shape=edges.shape[1])

# Crear (en formato tensowrflow) los features para cada nodo.
node_features = tf.cast(
    papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32
)

#### juan: esto se puede simplificar
# el grafo es la union de estas tres cosas
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)


### Esto es muy importante. 
### El primer vector es la lista de los indices de los nodos source de edges, 
### El segundo vector es la lista de los indices de los nodos target

node_indices, neighbour_indices = edges[0], edges[1]


Edges shape: (2, 5429)
Nodes shape: (2708, 1433)


/var/folders/gf/12lsjk9d5mn0gdyhlk_xd0zm0000gn/T/ipykernel_8662/1613730209.py:9: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32


### Un modelo para una capa de la GNN
La estructura de tensorflow nos obliga a definir estas operaciones, complejas, como otras layers. 

In [90]:
class GNNlayer(layers.Layer):
    def __init__(
        self,
        capas_internas,
        dropout_rate=0.2,
        normalize=False,
        *args,
        **kwargs,
    ):
        super(GNNlayer, self).__init__(*args, **kwargs)

        self.preprocesador = create_MLP(capas_internas, dropout_rate)

        self.updater = create_MLP(capas_internas, dropout_rate)

        
    def prepare(self, node_repesentations, weights=None):
        
#        Esta funcion pasa los mensajes por una red neuronal simple, y aplica los pesos (si hay)
        
        messages = self.preprocesador(node_repesentations)
        messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):
        
        # Esta funcion agrega los mensajes de cada nodo, en forma de suma. 
        # recibo un vector node_indices, que es de largo [num_edges] y me dice los nodos origen de cada arista
        # matriz neighbour_messages es de forma [num_edges, (neuronas_internas)], osea [num_edges, 32] en este codigo
        # esta matriz tiene el mensaje de cada nodo que participa en la arista como nodo destino
        # la matriz node_repesentations es de la forma [num_nodes, representation_dim], contiene información de 
        # los nodos del grafo. 
        
        num_nodes = node_repesentations.shape[0]
        
                
        #### La funcion unsorted_segment_sum me suma los mensajes de todos los indices iguales en node_indices y 
        #### los deja en el i-esimo lugar; con eso sumamos los ids. Si un nodo no tiene vecinos recibe un 0
        
        aggregated_message = tf.math.unsorted_segment_sum(
            neighbour_messages, node_indices, num_segments=num_nodes
        )
        
        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        
        # Para combinar los mensajes con los features de cada nodo, concatenamos. 
        # Notar que a este punto tanto node_repesentations como aggregated_messages tienen forma 
        # [num_nodes, representation_dim]. Concat me los concatena. 

        h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        
        # Y aplicamos unas capas no-lineales
        
        node_embeddings = self.updater(h)

        return node_embeddings

    def call(self, inputs):
        ## Procesa los inputs para crear los embeddings. Siempre tenemos información de todo el grafo, 
        ## y operamos sobre todos los nodos en node_representations

        node_repesentations, edges, edge_weights = inputs
        
        node_indices, neighbour_indices = edges[0], edges[1]
        
        # Lo primero es una lista de vectores en donde tomo cada id en neighbour_indices 
        # y lo reemplazo por la representación de ese id. 
        # El resultado es una lista que contiene, para cada arista, la representación del target de esa arista. 
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)

        # Procesamos estos mensajes (posiblemente incluyendo pesos en aristas)
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        
        #Los agregamos
        aggregated_messages = self.aggregate(
            node_indices, neighbour_messages, node_repesentations
        )
        
        # Y finalmente, el update. 
        return self.update(node_repesentations, aggregated_messages)


In [100]:
class GNNbasica(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        capas_internas = [32,32],
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        
        super(GNNbasica, self).__init__(*args, **kwargs)

#LA GNN maneja información de todo el grafo, independiente del batch que procese. 

        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        
        #normalizar
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

#Las layers básicas: una capa para preprocesar todo        
        self.preprocesar = create_MLP(capas_internas, dropout_rate, name="preprocesado")
    
# dos capas de paso de mensjaes 

        self.capa1 = GNNlayer(
            capas_internas,
            dropout_rate,
            name="capa1",
        )
        # Create the second GraphConv layer.
        self.capa2 = GNNlayer(
            capas_internas,
            dropout_rate,
            name="capa2",
        )
        
        # Create a postprocess layer.
        self.postprocess = create_MLP(capas_internas, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.clas = layers.Dense(units=num_classes, name="logits")

        
    def call(self, batch_indices):
        
        #### Capa de preprocesado de features, para bajar la dimensionalidad
        
        nodos_preprocesados = self.preprocesar(self.node_features)
        
        #### Estos nodos preprocesados pasan por la capa1, la que agrega los mensajes de sus vecinos. 
            
        paso_mens1 = self.capa1((nodos_preprocesados,self.edges, self.edge_weights))
        
        skip1 = nodos_preprocesados + paso_mens1 
        
        paso_mens2 = self.capa2((skip1,self.edges, self.edge_weights))
        
        skip2 = paso_mens2 + skip1

        ##### Postprocesado y llegar a la categoría del nodo
        postprocesado = self.postprocess(skip2)

        ##### Volvemos a poner los embeddings en el orden que demandaba segun el batch
        node_embeddings = tf.gather(postprocesado, batch_indices)

        # Readout para llegar a las categorias
        
        return self.clas(node_embeddings)
        

In [101]:

GNN = GNNbasica(
    graph_info=graph_info,
    num_classes=7,
    capas_internas=[32,32],
    dropout_rate=0.2,
    name="gnn_model",
)

print("GNN output shape:", GNN([1, 10, 100]))

GNN.summary()





GNN output shape: tf.Tensor(
[[ 0.00923411  0.00936708 -0.01251213  0.00217842 -0.00670045 -0.00551362
  -0.00453116]
 [ 0.01907693  0.00991904  0.01168425  0.00347327 -0.00653539  0.00136299
  -0.00352739]
 [ 0.01193399 -0.00514359  0.00655198  0.00696504  0.01629947 -0.01231707
  -0.02240883]], shape=(3, 7), dtype=float32)
Model: "gnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocesado (Sequential)   (2708, 32)                52804     
                                                                 
 capa1 (GNNlayer)            multiple                  5888      
                                                                 
 capa2 (GNNlayer)            multiple                  5888      
                                                                 
 postprocess (Sequential)    (2708, 32)                2368      
                                                            

In [102]:
GNN.compile(
        optimizer=keras.optimizers.Adam(0.01),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )

# A la GNN hay que darlos los ids de los nodos 

x_train = train_data.paper_id.to_numpy()
    # Fit the model.
history = GNN.fit(
        x=x_train,
        y=y_train,
        epochs=300,
        batch_size=256,
        validation_split=0.15,
        callbacks=[early_stopping],
    )



Epoch 1/300
5/5 [==============================] - 2s 99ms/step - loss: 1.9647 - acc: 0.2342 - val_loss: 1.9003 - val_acc: 0.2714
Epoch 2/300
5/5 [==============================] - 0s 40ms/step - loss: 1.8272 - acc: 0.2965 - val_loss: 1.8743 - val_acc: 0.2714
Epoch 3/300
5/5 [==============================] - 0s 41ms/step - loss: 1.7922 - acc: 0.3143 - val_loss: 1.8426 - val_acc: 0.3216
Epoch 4/300
5/5 [==============================] - 0s 42ms/step - loss: 1.6565 - acc: 0.3624 - val_loss: 1.7963 - val_acc: 0.3618
Epoch 5/300
5/5 [==============================] - 0s 40ms/step - loss: 1.5290 - acc: 0.4132 - val_loss: 1.6646 - val_acc: 0.4372
Epoch 6/300
5/5 [==============================] - 0s 39ms/step - loss: 1.4043 - acc: 0.4871 - val_loss: 1.5229 - val_acc: 0.4925
Epoch 7/300
5/5 [==============================] - 0s 45ms/step - loss: 1.2079 - acc: 0.5610 - val_loss: 1.4155 - val_acc: 0.4472
Epoch 8/300
5/5 [==============================] - 0s 44ms/step - loss: 0.9846 - acc: 0.64

5/5 [==============================] - 0s 48ms/step - loss: 0.0299 - acc: 0.9938 - val_loss: 1.7842 - val_acc: 0.6583
Epoch 64/300
5/5 [==============================] - 0s 44ms/step - loss: 0.0439 - acc: 0.9875 - val_loss: 1.6035 - val_acc: 0.6734
Epoch 65/300
5/5 [==============================] - 0s 42ms/step - loss: 0.0500 - acc: 0.9875 - val_loss: 1.7168 - val_acc: 0.6432
Epoch 66/300
5/5 [==============================] - 0s 44ms/step - loss: 0.0322 - acc: 0.9884 - val_loss: 1.6093 - val_acc: 0.6533
Epoch 67/300
5/5 [==============================] - 0s 46ms/step - loss: 0.0385 - acc: 0.9858 - val_loss: 1.9069 - val_acc: 0.6332
Epoch 68/300
5/5 [==============================] - 0s 45ms/step - loss: 0.0462 - acc: 0.9893 - val_loss: 2.0109 - val_acc: 0.6482
Epoch 69/300
5/5 [==============================] - 0s 45ms/step - loss: 0.0415 - acc: 0.9875 - val_loss: 1.5663 - val_acc: 0.6884
Epoch 70/300
5/5 [==============================] - 0s 46ms/step - loss: 0.0299 - acc: 0.9902 - 

5/5 [==============================] - 0s 47ms/step - loss: 0.0339 - acc: 0.9911 - val_loss: 1.0236 - val_acc: 0.8191
Epoch 126/300
5/5 [==============================] - 0s 44ms/step - loss: 0.0134 - acc: 0.9947 - val_loss: 1.1124 - val_acc: 0.7940
Epoch 127/300
5/5 [==============================] - 0s 46ms/step - loss: 0.0203 - acc: 0.9911 - val_loss: 1.1177 - val_acc: 0.7990
Epoch 128/300
5/5 [==============================] - 0s 46ms/step - loss: 0.0222 - acc: 0.9947 - val_loss: 1.1885 - val_acc: 0.8141
Epoch 129/300
5/5 [==============================] - 0s 50ms/step - loss: 0.0173 - acc: 0.9947 - val_loss: 1.2952 - val_acc: 0.8090
Epoch 130/300
5/5 [==============================] - 0s 43ms/step - loss: 0.0269 - acc: 0.9947 - val_loss: 1.2174 - val_acc: 0.8141
Epoch 131/300
5/5 [==============================] - 0s 44ms/step - loss: 0.0344 - acc: 0.9902 - val_loss: 1.0537 - val_acc: 0.8090
Epoch 132/300
5/5 [==============================] - 0s 45ms/step - loss: 0.0184 - acc: 0.

5/5 [==============================] - 0s 44ms/step - loss: 0.0119 - acc: 0.9955 - val_loss: 1.3954 - val_acc: 0.7889
Epoch 188/300
5/5 [==============================] - 0s 48ms/step - loss: 0.0149 - acc: 0.9955 - val_loss: 1.3835 - val_acc: 0.7990
Epoch 189/300
5/5 [==============================] - 0s 44ms/step - loss: 0.0168 - acc: 0.9955 - val_loss: 1.4706 - val_acc: 0.8090
Epoch 190/300
5/5 [==============================] - 0s 47ms/step - loss: 0.0155 - acc: 0.9947 - val_loss: 1.4832 - val_acc: 0.8040
Epoch 191/300
5/5 [==============================] - 0s 45ms/step - loss: 0.0287 - acc: 0.9911 - val_loss: 1.4440 - val_acc: 0.8141
Epoch 192/300
5/5 [==============================] - 0s 45ms/step - loss: 0.0191 - acc: 0.9955 - val_loss: 1.4309 - val_acc: 0.8040
Epoch 193/300
5/5 [==============================] - 0s 45ms/step - loss: 0.0121 - acc: 0.9955 - val_loss: 1.4359 - val_acc: 0.8090
Epoch 194/300
5/5 [==============================] - 0s 46ms/step - loss: 0.0150 - acc: 0.

In [103]:
x_test = test_data.paper_id.to_numpy()
GNN.evaluate(x=x_test, y=y_test)

44/44 [==============================] - 0s 9ms/step - loss: 1.3374 - acc: 0.7922


[1.3373955488204956, 0.7922077775001526]